# **Signal To Spike Conversion** - Analysis

This notebook conducts an analysis of the Inter-Spike Interval in the UP and DN spike trains to determine the optimal synaptic time constants

### Check WD (change if necessary) and file loading

In [331]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [332]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"results/{DATASET_FILENAME}"
RESULTS_FOLDER = f"analysis/{DATASET_FILENAME}"

## Load the UP and DN spike trains in the Ripple and FR Band

In [333]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import read_spike_events, MarkerType, band_to_file_name, BaselineAlgorithm

### Load the Baseline Thresholds from the output file from the baseline process

In [334]:
# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.Q3

In [335]:
# Load the Baseline Thresholds
BASELINE_FILE = f"baseline_results/{DATASET_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  3.7058 -3.7058
FR Thresholds:  1.4961 -1.4961


In [336]:
ripple_band_filename = band_to_file_name(MarkerType.RIPPLE)
fr_band_filename = band_to_file_name(MarkerType.FAST_RIPPLE)

# Call the function to read the spike events
ripple_up_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_up_spike_train_{ripple_thresh_up}.csv"
ripple_down_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_down_spike_train_{ripple_thresh_down}.csv"

ripple_up_spike_train = read_spike_events(ripple_up_spikes_file_path)
ripple_down_spike_train = read_spike_events(ripple_down_spikes_file_path)


preview_np_array(ripple_up_spike_train, "ripple_up_spike_train", edge_items=3)
preview_np_array(ripple_down_spike_train, "ripple_down_spike_train", edge_items=3)

ripple_up_spike_train Shape: (3724, 2).
Preview: [[ 9.43847656e+02 -1.00000000e+00]
 [ 9.51171875e+02 -1.00000000e+00]
 [ 9.52148438e+02 -1.00000000e+00]
 ...
 [ 1.19093262e+05 -1.00000000e+00]
 [ 1.19094727e+05 -1.00000000e+00]
 [ 1.19105957e+05 -1.00000000e+00]]
ripple_down_spike_train Shape: (3711, 2).
Preview: [[ 9.47265625e+02 -1.00000000e+00]
 [ 9.48242188e+02 -1.00000000e+00]
 [ 9.56054688e+02 -1.00000000e+00]
 ...
 [ 1.19089844e+05 -1.00000000e+00]
 [ 1.19098633e+05 -1.00000000e+00]
 [ 1.19100098e+05 -1.00000000e+00]]


In [337]:
fr_up_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_up_spike_train_{fr_thresh_up}.csv"
fr_down_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_down_spike_train_{fr_thresh_down}.csv"

fr_up_spike_train = read_spike_events(fr_up_spikes_file_path)
fr_down_spike_train = read_spike_events(fr_down_spikes_file_path)

preview_np_array(fr_up_spike_train, "fr_up_spike_train", edge_items=3)
preview_np_array(fr_down_spike_train, "fr_down_spike_train", edge_items=3)

fr_up_spike_train Shape: (9101, 2).
Preview: [[ 5.06347656e+02 -1.00000000e+00]
 [ 5.37109375e+02 -1.00000000e+00]
 [ 5.52246094e+02 -1.00000000e+00]
 ...
 [ 1.19975586e+05 -1.00000000e+00]
 [ 1.19979004e+05 -1.00000000e+00]
 [ 1.19981445e+05 -1.00000000e+00]]
fr_down_spike_train Shape: (9146, 2).
Preview: [[ 5.05371094e+02 -1.00000000e+00]
 [ 5.36132812e+02 -1.00000000e+00]
 [ 5.51269531e+02 -1.00000000e+00]
 ...
 [ 1.19959961e+05 -1.00000000e+00]
 [ 1.19976562e+05 -1.00000000e+00]
 [ 1.19980469e+05 -1.00000000e+00]]


### Load the Annotated Data
The annotated data is needed to separate the intervals of HFO from the normal activity to calculate the Inter-Spike Interval (ISI)

In [338]:
# load the npy object
ripple_ground_truth = np.load(f"{INPUT_FOLDER}/{ripple_band_filename}_ground_truth.npy")
fr_ground_truth = np.load(f"{INPUT_FOLDER}/{fr_band_filename}_ground_truth.npy")

preview_np_array(ripple_ground_truth, "ripple_ground_truth", edge_items=3)
preview_np_array(fr_ground_truth, "fr_ground_truth", edge_items=3)

ripple_ground_truth Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]
fr_ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


## Iterate the UP and DN Spike Trains to Calculate the Inter-Spike Interval (ISI) in the HFO and Normal Activity

Define the lists to store:
- `Ripple_HFO_UP` - ISI of the UP train in the Ripple band during an HFO event 
- `Ripple_HFO_DN` - ISI of the DN train in the Ripple band during an HFO event
- `Ripple_Noise_UP` - ISI of the UP train in the Ripple band during normal activity
- `Ripple_Noise_DN` - ISI of the DN train in the Ripple band during normal activity

- `FR_HFO_UP` - ISI of the UP train in the Fast Ripple band during an HFO event
- `FR_HFO_DN` - ISI of the DN train in the Fast Ripple band during an HFO event
- `FR_Noise_UP` - ISI of the UP train in the Fast Ripple band during normal activity
- `FR_Noise_DN` - ISI of the DN train in the Fast Ripple band during normal activity

## Let's analyze the Ripple Band first

In [339]:
isi_ripple_hfo_up = []
isi_ripple_hfo_down = []
isi_ripple_noise_up = []
isi_ripple_noise_down = []

#### Ripple Band - UP Train

In [340]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Iterate the Ripple UP Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_up_spike_train), 1):
    spike_time = ripple_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1

    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW

            print(f"Spike time: {spike_time}. ISI: {isi}. is_first_spike: {is_first_spike}. different_hfo: {different_hfo}")
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

Spike time: 1013.18359375. ISI: 20.01953125. is_first_spike: True. different_hfo: False
Spike time: 1017.578125. ISI: 4.39453125. is_first_spike: False. different_hfo: False
Spike time: 3208.984375. ISI: 9.765625. is_first_spike: True. different_hfo: False
Spike time: 3220.703125. ISI: 11.71875. is_first_spike: False. different_hfo: False
Spike time: 3221.19140625. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 3222.16796875. ISI: 0.9765625. is_first_spike: False. different_hfo: False
Spike time: 3232.421875. ISI: 10.25390625. is_first_spike: False. different_hfo: False
Spike time: 3232.91015625. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 3233.3984375. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 3241.2109375. ISI: 7.8125. is_first_spike: False. different_hfo: False
Spike time: 3242.67578125. ISI: 1.46484375. is_first_spike: False. different_hfo: False
Spike time: 3243.65234375. ISI: 0.9765625. is_first

In [341]:
print(f"Max ISI Ripple UP HFO: {max(isi_ripple_hfo_up)}")
print(f"Min ISI Ripple UP HFO: {min(isi_ripple_hfo_up)}")

Max ISI Ripple UP HFO: 112.3046875
Min ISI Ripple UP HFO: 0.48828125


#### Ripple Band - DOWN Train

In [342]:
# Iterate the Ripple DOWN Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_down_spike_train), 1):
    spike_time = ripple_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_down.append(isi)
            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

In [343]:
print(f"Max ISI Ripple DOWN HFO: {max(isi_ripple_hfo_down)}")
print(f"Min ISI Ripple DOWN HFO: {min(isi_ripple_hfo_down)}")

Max ISI Ripple DOWN HFO: 93.26171875
Min ISI Ripple DOWN HFO: 0.48828125


### Validate the number of calculated ISIs
Validate that:
- `len(isi_ripple_hfo_up)` + `len(isi_ripple_noise_up)` = `len(ripple_up)` - 1 (first spike does not have an ISI)
- `len(isi_ripple_hfo_down_)` + `len(isi_ripple_noise_down_)` = `len(ripple_down_)` - 1 (first spike does not have an ISI)

In [344]:
#  Validate that len(isi_ripple_hfo_up) + len(isi_ripple_noise_up) = len(ripple_up)
print(f"Length isi_ripple_hfo_up: {len(isi_ripple_hfo_up)}")
print(f"Length isi_ripple_noise_up: {len(isi_ripple_noise_up)}")
print(f"Sum: {len(isi_ripple_hfo_up) + len(isi_ripple_noise_up)}")
print(f"Length ripple_up: {len(ripple_up_spike_train)}")

print(f"{round(len(isi_ripple_hfo_up)/len(ripple_up_spike_train)*100, 2)}% of the UP spikes are inside the HFO")

Length isi_ripple_hfo_up: 2561
Length isi_ripple_noise_up: 1162
Sum: 3723
Length ripple_up: 3724
68.77% of the UP spikes are inside the HFO


In [345]:
#  Validate that len(isi_ripple_hfo_down) + len(isi_ripple_noise_down) = len(ripple_down)
print(f"Length isi_ripple_hfo_down: {len(isi_ripple_hfo_down)}")
print(f"Length isi_ripple_noise_down: {len(isi_ripple_noise_down)}")
print(f"Sum: {len(isi_ripple_hfo_down) + len(isi_ripple_noise_down)}")
print(f"Length ripple_down: {len(ripple_down_spike_train)}")

print(f"{round(len(isi_ripple_hfo_down)/len(ripple_down_spike_train)*100, 2)}% of the DOWN spikes are inside the HFO")

Length isi_ripple_hfo_down: 2526
Length isi_ripple_noise_down: 1184
Sum: 3710
Length ripple_down: 3711
68.07% of the DOWN spikes are inside the HFO


## Let's analyze the Fast Ripple Band next

In [346]:
isi_fr_hfo_up = []
isi_fr_hfo_down = []
isi_fr_noise_up = []
isi_fr_noise_down = []

#### Fast Ripple Band - UP Train

In [347]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_up_spike_train), 1):
    spike_time = fr_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

#### Fast Ripple Band - DN Train

In [348]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_down_spike_train), 1):
    spike_time = fr_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            
            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

### Validate the number of calculated ISIs
Validate that:
- `len(isi_fr_hfo_up)` + `len(isi_fr_noise_up)` = `len(fr_up)` - 1 (first spike does not have an ISI)
- `len(isi_fr_hfo_down)` + `len(isi_fr_noise_down)` = `len(fr_down)` - 1 (first spike does not have an ISI)

In [349]:
#  Validate that len(isi_fr_hfo_up) + len(isi_fr_noise_up) = len(ripple_up)
print(f"Length isi_fr_hfo_up: {len(isi_fr_hfo_up)}")
print(f"Length isi_fr_noise_up: {len(isi_fr_noise_up)}")
print(f"Sum: {len(isi_fr_hfo_up) + len(isi_fr_noise_up)}")
print(f"Length fr_up: {len(fr_up_spike_train)}")

print(f"{round(len(isi_fr_hfo_up)/len(fr_up_spike_train)*100, 2)}% of the FR UP spikes are inside the HFO")

Length isi_fr_hfo_up: 3089
Length isi_fr_noise_up: 6011
Sum: 9100
Length fr_up: 9101
33.94% of the FR UP spikes are inside the HFO


In [350]:
#  Validate that len(isi_fr_hfo_down) + len(isi_fr_noise_down) = len(ripple_down)
print(f"Length isi_fr_hfo_down: {len(isi_fr_hfo_down)}")
print(f"Length isi_fr_noise_down: {len(isi_fr_noise_down)}")
print(f"Sum: {len(isi_fr_hfo_down) + len(isi_fr_noise_down)}")
print(f"Length fr_down: {len(fr_down_spike_train)}")

print(f"{round(len(isi_fr_hfo_down)/len(fr_down_spike_train)*100, 2)}% of the FR DOWN spikes are inside the HFO")

Length isi_fr_hfo_down: 3141
Length isi_fr_noise_down: 6004
Sum: 9145
Length fr_down: 9146
34.34% of the FR DOWN spikes are inside the HFO


--- 

## Calculate metrics of the ISIs and Show the Results

In [351]:
# Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_ripple_hfo_up = np.mean(isi_ripple_hfo_up)
median_ripple_hfo_up = np.median(isi_ripple_hfo_up)
std_ripple_hfo_up = np.std(isi_ripple_hfo_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during HFO
q1_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 25)
q3_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 75)
iqr_ripple_hfo_up = q3_ripple_hfo_up - q1_ripple_hfo_up

# ---- Baseline Activity Metrics ----
mean_ripple_noise_up = np.mean(isi_ripple_noise_up)
median_ripple_noise_up = np.median(isi_ripple_noise_up)
std_ripple_noise_up = np.std(isi_ripple_noise_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during Noise
q1_ripple_noise_up = np.percentile(isi_ripple_noise_up, 25)
q3_ripple_noise_up = np.percentile(isi_ripple_noise_up, 75)
iqr_ripple_noise_up = q3_ripple_noise_up - q1_ripple_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_ripple_hfo_down = np.mean(isi_ripple_hfo_down)
median_ripple_hfo_down = np.median(isi_ripple_hfo_down)
std_ripple_hfo_down = np.std(isi_ripple_hfo_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during HFO
q1_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 25)
q3_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 75)
iqr_ripple_hfo_down = q3_ripple_hfo_down - q1_ripple_hfo_down

# ---- Baseline Activity Metrics ----
mean_ripple_noise_down = np.mean(isi_ripple_noise_down)
median_ripple_noise_down = np.median(isi_ripple_noise_down)
std_ripple_noise_down = np.std(isi_ripple_noise_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during Noise
q1_ripple_noise_down = np.percentile(isi_ripple_noise_down, 25)
q3_ripple_noise_down = np.percentile(isi_ripple_noise_down, 75)
iqr_ripple_noise_down = q3_ripple_noise_down - q1_ripple_noise_down

In [352]:
# Fast Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_fr_hfo_up = np.mean(isi_fr_hfo_up)
median_fr_hfo_up = np.median(isi_fr_hfo_up)
std_fr_hfo_up = np.std(isi_fr_hfo_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during HFO
q1_fr_hfo_up = np.percentile(isi_fr_hfo_up, 25)
q3_fr_hfo_up = np.percentile(isi_fr_hfo_up, 75)
iqr_fr_hfo_up = q3_fr_hfo_up - q1_fr_hfo_up

# ---- Baseline Activity Metrics ----
mean_fr_noise_up = np.mean(isi_fr_noise_up)
median_fr_noise_up = np.median(isi_fr_noise_up)
std_fr_noise_up = np.std(isi_fr_noise_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during Noise
q1_fr_noise_up = np.percentile(isi_fr_noise_up, 25)
q3_fr_noise_up = np.percentile(isi_fr_noise_up, 75)
iqr_fr_noise_up = q3_fr_noise_up - q1_fr_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_fr_hfo_down = np.mean(isi_fr_hfo_down)
median_fr_hfo_down = np.median(isi_fr_hfo_down)
std_fr_hfo_down = np.std(isi_fr_hfo_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during HFO
q1_fr_hfo_down = np.percentile(isi_fr_hfo_down, 25)
q3_fr_hfo_down = np.percentile(isi_fr_hfo_down, 75)
iqr_fr_hfo_down = q3_fr_hfo_down - q1_fr_hfo_down

# ---- Baseline Activity Metrics ----
mean_fr_noise_down = np.mean(isi_fr_noise_down)
median_fr_noise_down = np.median(isi_fr_noise_down)
std_fr_noise_down = np.std(isi_fr_noise_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during Noise
q1_fr_noise_down = np.percentile(isi_fr_noise_down, 25)
q3_fr_noise_down = np.percentile(isi_fr_noise_down, 75)
iqr_fr_noise_down = q3_fr_noise_down - q1_fr_noise_down

### Print the Metrics of the ISIs

#### Ripple Band

In [353]:
# Metrics of the ISIs in the Ripple Band
print("Ripple UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_up}, Median: {median_ripple_hfo_up}, STD: {std_ripple_hfo_up}, IQR: [{q1_ripple_hfo_up} - {q3_ripple_hfo_up}], Max: {max(isi_ripple_hfo_up)}, Min: {min(isi_ripple_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_up}, Median: {median_ripple_noise_up}, STD: {std_ripple_noise_up}, IQR: [{q1_ripple_noise_up} - {q3_ripple_noise_up}], Max: {max(isi_ripple_noise_up)}, Min: {min(isi_ripple_noise_up)}")

print("=================================")
print("Ripple DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_down}, Median: {median_ripple_hfo_down}, STD: {std_ripple_hfo_down}, IQR: [{q1_ripple_hfo_down} - {q3_ripple_hfo_down}], Max: {max(isi_ripple_hfo_down)}, Min: {min(isi_ripple_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_down}, Median: {median_ripple_noise_down}, STD: {std_ripple_noise_down}, IQR: [{q1_ripple_noise_down}  - {q3_ripple_noise_down}], Max: {max(isi_ripple_noise_down)}, Min: {min(isi_ripple_noise_down)}")

Ripple UP
HFO ISI (ms)
Mean: 3.8734564135103473, Median: 0.9765625, STD: 7.495783076659881, IQR: [0.48828125 - 5.859375], Max: 112.3046875, Min: 0.48828125
Noise ISI (ms)
Mean: 93.15162435456111, Median: 12.20703125, STD: 170.94027240744822, IQR: [5.859375 - 110.107421875], Max: 2193.84765625, Min: 0.48828125
Ripple DOWN
HFO ISI (ms)
Mean: 3.8001271154988125, Median: 0.9765625, STD: 7.142910347352718, IQR: [0.48828125 - 5.37109375], Max: 93.26171875, Min: 0.48828125
Noise ISI (ms)
Mean: 91.68387748099663, Median: 12.20703125, STD: 170.25909818093413, IQR: [5.37109375  - 106.0791015625], Max: 2194.3359375, Min: 0.48828125


#### Fast Ripple Band

In [354]:
# Metrics of the ISIs in the FR Band
print("FR UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_up}, Median: {median_fr_hfo_up}, STD: {std_fr_hfo_up}, IQR: [{q1_fr_hfo_up} - {q3_fr_hfo_up}], Max: {max(isi_fr_hfo_up)}, Min: {min(isi_fr_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_up}, Median: {median_fr_noise_up}, STD: {std_fr_noise_up}, IQR: [{q1_fr_noise_up} - {q3_fr_noise_up}], Max: {max(isi_fr_noise_up)}, Min: {min(isi_fr_noise_up)}")

print("=================================")
print("FR DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_down}, Median: {median_fr_hfo_down}, STD: {std_fr_hfo_down}, IQR: [{q1_fr_hfo_down} - {q3_fr_hfo_down}], Max: {max(isi_fr_hfo_down)}, Min: {min(isi_fr_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_down}, Median: {median_fr_noise_down}, STD: {std_fr_noise_down}, IQR: [{q1_fr_noise_down} - {q3_fr_noise_down}], Max: {max(isi_fr_noise_down)}, Min: {min(isi_fr_noise_down)}")

FR UP
HFO ISI (ms)
Mean: 2.1587753419391387, Median: 0.48828125, STD: 3.366158468643759, IQR: [0.48828125 - 2.9296875], Max: 40.52734375, Min: 0.48828125
Noise ISI (ms)
Mean: 18.766701152054566, Median: 3.90625, STD: 48.66777871129274, IQR: [2.44140625 - 17.08984375], Max: 958.49609375, Min: 0.48828125
FR DOWN
HFO ISI (ms)
Mean: 2.151484399872652, Median: 0.9765625, STD: 3.4324468117417686, IQR: [0.48828125 - 2.9296875], Max: 48.33984375, Min: 0.48828125
Noise ISI (ms)
Mean: 18.773698393779146, Median: 3.90625, STD: 48.77782218092905, IQR: [1.953125 - 17.08984375], Max: 958.49609375, Min: 0.48828125


### Find number of outliers in the ISIs

In [355]:
# Print the number of outliers in the ISIs
# Ripple UP
ripple_upper_outliers = len([curr_isi for curr_isi in isi_ripple_hfo_up if curr_isi > q3_ripple_hfo_up + 1.5 * iqr_ripple_hfo_up])

print("Ripple UP HFO Outliers")
print(f"Upper Outliers: {ripple_upper_outliers}/{len(isi_ripple_hfo_up)} ({round(ripple_upper_outliers/len(isi_ripple_hfo_up) * 100, 2)}%)")

# FR UP
fr_upper_outliers = len([curr_isi for curr_isi in isi_fr_hfo_up if curr_isi > q3_fr_hfo_up + 1.5 * iqr_fr_hfo_up])

print("FR UP HFO Outliers")
print(f"Upper Outliers: {fr_upper_outliers}/{len(isi_fr_hfo_up)} ({round(fr_upper_outliers/len(isi_fr_hfo_up) * 100, 2)}%)")

Ripple UP HFO Outliers
Upper Outliers: 53/2561 (2.07%)
FR UP HFO Outliers
Upper Outliers: 130/3089 (4.21%)


### Display Box Plots with the Inter-Spike Intervals

#### Ripple ISIs

In [356]:
# Display Box Plots with the Inter-Spike Intervals
from utils.bar_plot import create_box_plot
import bokeh.plotting as bplt

# Ripple Band Box Plot
# Define the array of values
ripple_isi_list = [isi_ripple_hfo_up, isi_ripple_hfo_down, isi_ripple_noise_up, isi_ripple_noise_down]
# Create the BoxPlot
ripple_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band", 
    box_arrays=ripple_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_ripple_isi_barplot = True
if show_ripple_isi_barplot:
    # Show the plot
    bplt.show(ripple_isi_boxplot)

ripple_quantiles:  [0.48828125 0.9765625  5.859375  ]
Ripple IQR:  5.37109375
ripple_quantiles:  [0.48828125 0.9765625  5.37109375]
Ripple IQR:  4.8828125
ripple_quantiles:  [  5.859375    12.20703125 110.10742188]
Ripple IQR:  104.248046875
ripple_quantiles:  [  5.37109375  12.20703125 106.07910156]
Ripple IQR:  100.7080078125


In [357]:
# Export the plot to an HTML file
EXPORT_RIPPLE_BOXPLOT = True
if EXPORT_RIPPLE_BOXPLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    ripple_boxplot_filename = f"{RESULTS_FOLDER}/ripple_isi_boxplot_thresh{ripple_thresh_up}-{ripple_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=ripple_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band")

    # Save the plot
    bplt.save(ripple_isi_boxplot)

    # Close the plot
    bplt.reset_output()

#### Fast Ripple ISIs

In [358]:
# Display Box Plots with the Inter-Spike Intervals

# FR Band Box Plot
# Define the array of values
fr_isi_list = [isi_fr_hfo_up, isi_fr_hfo_down, isi_fr_noise_up, isi_fr_noise_down]
# Create the BoxPlot
fr_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band", 
    box_arrays=fr_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_fr_isi_barplot = True
if show_fr_isi_barplot:
    # Show the plot
    bplt.show(fr_isi_boxplot)

ripple_quantiles:  [0.48828125 0.48828125 2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [0.48828125 0.9765625  2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [ 2.44140625  3.90625    17.08984375]
Ripple IQR:  14.6484375
ripple_quantiles:  [ 1.953125    3.90625    17.08984375]
Ripple IQR:  15.13671875


In [359]:
# Export the plot to an HTML file
EXPORT_FR_PLOT = True
if EXPORT_FR_PLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    fr_boxplot_filename = f"{RESULTS_FOLDER}/fr_isi_boxplot_thresh{fr_thresh_up}-{fr_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=fr_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band")

    # Save the plot
    bplt.save(fr_isi_boxplot)

    bplt.reset_output()